In [0]:
#LIBRERÍAS 
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
#solo p google colab
from google.colab import files


In [9]:
#DESCARGAR LOS DATOS, DESEMPAQUETAR Y GUARDAR
zipurl ="http://187.191.75.115/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip"
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('C://Users//JESICA//C3//Datos COVID DIARIOS')
        nombrearchivo=zfile.namelist()
        print (nombrearchivo[0])

200420COVID19MEXICO.csv


In [0]:
# CARGAR DATOS FEDERALES Y GEOGRÁFICOS
types_dict = {'ENTIDAD_RES': object,'MUNICIPIO_RES': object,'FECHA_DEF':str,'INTUBADO':int,'NEUMONIA':int,'EMBARAZO':int,'DIABETES':int,'EPOC':int,'ASMA':int,'INMUSUPR':int,'HIPERTENSION':int,'OTRA_COM':int,'CARDIOVASCULAR':int,'OBESIDAD':int,'RENAL_CRONICA':int,'TABAQUISMO':int,'OTRO_CASO':int,'UCI':int}
#si lo descargas de internet correr
#datosfederalCOVID19=pd.read_csv(nombrearchivo[0],encoding="LATIN1",dtype=types_dict)
#correr si tiene el archivo en local
datosfederalCOVID19=pd.read_csv("200420COVID19MEXICO.csv",encoding="LATIN1",dtype=types_dict)
#DATOS GEOGRÁFICOS en local
geograficosMUN=pd.read_excel("municipioscentroide.xlsx",encoding="LATIN1")
geograficosEDO=pd.read_excel("Geográficos.xlsx",encoding="LATIN1")

In [0]:
#LIMPIEZA CLAVES ESTADO
def fix_trunc_zeros(val):
    if len(str(val)) == 1:
        return "0"+str(val)
    else:
        # Return the value itself
        return str(val)
#LIMPIEZA ESTADOS DATOS GEOGRÁFICOS Y DE COVID
datosfederalCOVID19["ENTIDAD_RES"] = datosfederalCOVID19["ENTIDAD_RES"].apply(lambda x: fix_trunc_zeros(x))
geograficosMUN["CVE_ENT"] = geograficosMUN["CVE_ENT"].apply(lambda x: fix_trunc_zeros(x))
geograficosEDO["CLAVE_ENTIDAD"] = geograficosEDO["CLAVE_ENTIDAD"].apply(lambda x: fix_trunc_zeros(x))

In [0]:
#LIMPIEZA CLAVES MUNICIPIOS
def fix_trunc_zeros(val):
    if len(str(val)) == 1:
        return "00"+str(val)
    elif len(str(val)) == 2:
      return "0"+str(val)   
    else:
        # Return the value itself
        return str(val)
#LIMPIEZA MUNICIPIOS        
datosfederalCOVID19["MUNICIPIO_RES"] = datosfederalCOVID19["MUNICIPIO_RES"].apply(lambda x: fix_trunc_zeros(x))
geograficosMUN["CVE_MUN"] = geograficosMUN["CVE_MUN"].apply(lambda x: fix_trunc_zeros(x))

In [0]:
#CREACIÓN CLAVE ÚNICA MUNICIPIOS LLAVE PARA COMBINACION
datosfederalCOVID19["cveunica"]=datosfederalCOVID19["ENTIDAD_RES"]+datosfederalCOVID19["MUNICIPIO_RES"]

In [0]:
#LIMPIEZA CLAVES UNICA
def fix_trunc_zeros(val):
    if len(str(val)) == 4:
         return "0"+str(val)
    elif len(str(val)) == 3:
         return "00"+str(val) 
    elif len(str(val)) == 2:
        return "000"+str(val)
    elif len(str(val)) == 1:
        return "0000"+str(val)    
    else:
        # Return the value itself
        return str(val)
#LIMPIEZA CVEUNICA GEOG
geograficosMUN['cveunica'] =geograficosMUN['cveunica'].apply(lambda x: fix_trunc_zeros(x))
datosfederalCOVID19["cveunica"] =datosfederalCOVID19["cveunica"].apply(lambda x: fix_trunc_zeros(x))

In [0]:
#REORDENAR COLUMNAS
datosfederalCOVID19=datosfederalCOVID19[['FECHA_ACTUALIZACION','FECHA_INGRESO', 'FECHA_SINTOMAS','FECHA_DEF','ORIGEN', 'SECTOR', 'ENTIDAD_UM','ENTIDAD_NAC','ENTIDAD_RES', 'MUNICIPIO_RES','cveunica','MIGRANTE','NACIONALIDAD', 'PAIS_NACIONALIDAD','PAIS_ORIGEN', 'EDAD','SEXO','HABLA_LENGUA_INDIG','RESULTADO','TIPO_PACIENTE','INTUBADO', 'NEUMONIA','EMBARAZO','DIABETES','EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM','CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO','OTRO_CASO','UCI']]

In [0]:
#CAMBIAR A MINUSCULAS LOS NCOLUMNAPARA ARCVIVOS CSV
geograficosEDO =geograficosEDO.rename(columns={"CLAVE_ENTIDAD":"entidad_res"})
datosfederalCOVID19.columns = map(str.lower, datosfederalCOVID19.columns)

In [0]:
#CREAR COLUMNA DECESOS
datosfederalCOVID19['fecha_def'] = datosfederalCOVID19['fecha_def'].astype(str)

In [0]:
#FUNCION PARA COLUMNA DECESOS
def decesos(val):
    if val=="9999-99-99":
          return "0"  
    else:
        # Return the value itself
        return "1"
datosfederalCOVID19["decesos"]=datosfederalCOVID19['fecha_def'].apply(lambda x: decesos(x))

In [0]:
#CALCULO ESPECIE totalpruebas municipio se consideran todos los sospechosos ---"resultado"1,2,3
df=datosfederalCOVID19.groupby(["entidad_res","municipio_res","cveunica"])[("resultado")].value_counts().unstack().fillna(0)
df[1]= df[1].fillna(0)
df[2]= df[2].fillna(0)
df[3]= df[3].fillna(0)
df["Pruebastotales"]=df[1]+df[2]+df[3]
df.reset_index(inplace=True)
df=pd.merge(df,geograficosMUN,on="cveunica",how="right")
labels=["1","2","3","4","5","6","7","8","9","10"]
df["Pruebastotales"]= df["Pruebastotales"].fillna(0)
df["ordenvalido"]="Totales"
df["clasevalida"]="Datos_abiertos"
df["familiavalida"]="Incidencias"
df["diacolecta"]="1"
df["mescolecta"]="1"
df["aniocolecta"]="2010"
df["idejemplar"]="COVID-19"
df["generovalido"]="Pruebastotalespmunicipio"
df["paismapa"]="México"
df["altitudmapa"]="9999"
df["reinovalido"]="Epidemiologicos"
df["phylumdivisionvalido"]="Dirección General de Epidemiología"
df["categoriainfraespecievalida"]="total_municipio_pruebastotales"
df["proyecto"]="EPI-SPECIES"
df["calculoespecievalida"]=pd.cut(df["Pruebastotales"],10)
#CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
 #GUARDAMOS NUESTRO DATAFRAME SELECCIONANDO LAS VARIABLES DE INTERÉS
dfmunicipios=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
 #NOMBRAMOS CADA DATAFRAME
dfmunicipios.to_csv("datosMPOpruebastotales.csv",encoding="UTF-8",index=False)
 #DESCARGAMOS CADA DATAFRAME #solo en google colab, local descarga directo
files.download("datosMPOpruebastotales.csv")
#CAMPOS 100kh
df["generovalido"]="pruebastotalespmunicipiopor100,000hab"
df["valores"]=df["Pruebastotales"]/df["Población"]*100000
df["calculoespecievalida"]=pd.cut(df["valores"],10)
#CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
dfmunckhab=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
dfmunckhab.to_csv("datosMPOpruebastotalesp100kh.csv",encoding="UTF-8",index=False)
#DESCARGAMOS CADA DATAFRAME#solo en google colab, local descarga directo
files.download("datosMPOpruebastotalesp100kh.csv")

In [0]:
#CALCULO ESPECIE Totalpruebas Estado = todos los sospechosos ---resultado1,2,3
df=datosfederalCOVID19.groupby(["entidad_res"])[("resultado")].value_counts().unstack().fillna(0)
df[1]= df[1].fillna(0)
df[2]= df[2].fillna(0)
df[3]= df[3].fillna(0)
df["Pruebastotales"]=df[1]+df[2]+df[3]
df.reset_index(inplace=True)
df=pd.merge(df,geograficosEDO,on="entidad_res",how="right")
labels=["1","2","3","4","5","6","7","8","9","10"]
df["Pruebastotales"]= df["Pruebastotales"].fillna(0)
df["ordenvalido"]="Totales"
df["clasevalida"]="Datos_abiertos"
df["familiavalida"]="Incidencias"
df["diacolecta"]="1"
df["mescolecta"]="1"
df["aniocolecta"]="2010"
df["idejemplar"]="COVID-19"
df["generovalido"]="Pruebastotalespestado"
df["paismapa"]="México"
df["altitudmapa"]="9999"
df["reinovalido"]="Epidemiologicos"
df["phylumdivisionvalido"]="Dirección General de Epidemiología"
df["categoriainfraespecievalida"]="total_estado_pruebastotales"
df["proyecto"]="EPI-SPECIES"     
df["calculoespecievalida"]=pd.cut(df["Pruebastotales"],10)
#CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
#GUARDAMOS NUESTRO DATAFRAME SELECCIONANDO LAS VARIABLES DE INTERÉS
dfEDO=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
#NOMBRAMOS CADA DATAFRAME
dfEDO.to_csv("datosEDOpruebastotales.csv",encoding="UTF-8",index=False)
#DESCARGAMOS CADA DATAFRAME SOLO EN COLAB, EN LOCAL ES AUTOMÁTICO
files.download("datosEDOpruebastotales.csv")
df["generovalido"]="pruebastotalesedo100,000hab"
df["valores"]=df["Pruebastotales"]/df["Población"]*100000
df["calculoespecievalida"]=pd.cut(df["valores"],10)
#CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
dfEDOkhab=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
dfEDOkhab.to_csv("datosEDOpruebastotalesp100kh.csv",encoding="UTF-8",index=False)
#DESCARGAMOS CADA DATAFRAMESOLO EN COLAB, EN LOCAL ES AUTOMÁTICO
files.download("datosEDOpruebastotalesp100kh.csv")

In [0]:
#CALCULO ESPECIE hospitalizadostodos municipio= resultado1,2,3 y tipo paciente 2
df=datosfederalCOVID19.groupby(["entidad_res","municipio_res","cveunica"])[("tipo_paciente")].value_counts().unstack().fillna(0)
df[2]= df[2].fillna(0)
df.reset_index(inplace=True)
df=pd.merge(df,geograficosMUN,on="cveunica",how="right")
labels=["1","2","3","4","5","6","7","8","9","10"]
df[2]= df[2].fillna(0)
df["hospitalizados"]= df[2]
df["ordenvalido"]="Totales"
df["clasevalida"]="Datos_abiertos"
df["familiavalida"]="Incidencias"
df["diacolecta"]="1"
df["mescolecta"]="1"
df["aniocolecta"]="2010"
df["idejemplar"]="COVID-19"
df["generovalido"]="Hospitalizadostotalespmunicipio"
df["paismapa"]="México"
df["altitudmapa"]="9999"
df["reinovalido"]="Epidemiologicos"
df["phylumdivisionvalido"]="Dirección General de Epidemiología"
df["categoriainfraespecievalida"]="total_municipio_hospitalizados"
df["proyecto"]="EPI-SPECIES"    
df["calculoespecievalida"]=pd.cut(df["hospitalizados"],10)
#CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
 #GUARDAMOS NUESTRO DATAFRAME SELECCIONANDO LAS VARIABLES DE INTERÉS
dfmunicipios=df.loc[:, 'estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
 #NOMBRAMOS CADA DATAFRAME
dfmunicipios.to_csv("datosMPOhospitalizadostotales.csv",encoding="UTF-8",index=False)
 #DESCARGAMOS CADA DATAFRAME
files.download("datosMPOhospitalizadostotales.csv")
df["generovalido"]="datosMPOhospitalizadostotalespor100,000hab"
df["valores"]=df["hospitalizados"]/df["Población"]*100000
df["calculoespecievalida"]=pd.cut(df["valores"],10)
#CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
dfmunckhab=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
dfmunckhab.to_csv("datosMPOhospitalizadostotalespor100,000hab.csv",encoding="UTF-8",index=False)
#DESCARGAMOS CADA DATAFRAME
files.download("datosMPOhospitalizadostotalespor100,000hab.csv")

In [0]:
#CALCULO ESPECIE VFhospitalizadostodos Estado=  resultado1,2,3 y tipo paciente 2
df=datosfederalCOVID19.groupby(["entidad_res"])[("tipo_paciente")].value_counts().unstack().fillna(0)
df[2]= df[2].fillna(0)
df.reset_index(inplace=True)
df=pd.merge(df,geograficosEDO,on="entidad_res",how="right")
labels=["1","2","3","4","5","6","7","8","9","10"]
df[2]= df[2].fillna(0)
df["hospitalizados"]=df[2]
df["ordenvalido"]="Totales"
df["clasevalida"]="Datos_abiertos"
df["familiavalida"]="Incidencias"
df["diacolecta"]="1"
df["mescolecta"]="1"
df["aniocolecta"]="2010"
df["idejemplar"]="COVID-19"
df["generovalido"]="Hospitalizadostotalespestado"
df["paismapa"]="México"
df["altitudmapa"]="9999"
df["reinovalido"]="Epidemiologicos"
df["phylumdivisionvalido"]="Dirección General de Epidemiología"
df["categoriainfraespecievalida"]="total_estado_hospitalizados"
df["proyecto"]="EPI-SPECIES"
df["calculoespecievalida"]=pd.cut(df["hospitalizados"],10)
#CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
 #GUARDAMOS NUESTRO DATAFRAME SELECCIONANDO LAS VARIABLES DE INTERÉS
dfEDO=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
#NOMBRAMOS CADA DATAFRAME
dfEDO.to_csv("datosEDOhospitalizadostotales.csv",encoding="UTF-8",index=False)
#DESCARGAMOS CADA DATAFRAME
files.download("datosEDOhospitalizadostotales.csv")
df["generovalido"]="pruebastotalesedo100,000hab"
df["valores"]=df["hospitalizados"]/df["Población"]*100000
df["calculoespecievalida"]=pd.cut(df["valores"],10)
#CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
dfEDOkhab=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
dfEDOkhab.to_csv("datosEDOhospitalizadostotalesp100kh.csv",encoding="UTF-8",index=False)
#DESCARGAMOS CADA DATAFRAME
files.download("datosEDOhospitalizadostotalesp100kh.csv")

In [54]:
#LOOP FINAL ESPECIES COMORBILIDAD MUNICIPIOS TOTALES COMENTADO PARA TODAS LAS VARIABLES MORBILIDAD CONFIRMADOS COVID
#CON LOS DATOS FEDERALES Y GEOGRAFICOS LIMPIOS SELECCIONAMOS LAS COLUMNAS CON VARIABLES DE MORBILIDAD Y DECESOS
for i in datosfederalCOVID19.columns[35:36]:
   print (i)
   #HACEMOS LA AGRUPACION POR ENTIDAD,MUJNICIPIO,CVEUNICADEMUN,RESULTADO Y CONTAMOS POR LA COLUMNA DE MORBILIDAD
   df=datosfederalCOVID19.groupby(["entidad_res","municipio_res","cveunica","resultado"])[(i)].value_counts().unstack().fillna(0)
   #CONVERTIMOS A UN DATAFRAME LOS RESULTADOS DEL GROUPBY
   df.reset_index(inplace=True)
   #AQUI SELECCIONAMOS LOS CONFIRMADOS "1" O LOS QUE ESTÁN EN ESPERA DE ACUERDO AL RESULTADO DE LA PRUEBA "3"
   df=df.loc[df.loc[:,"resultado"] == 1]
   #JUNTAMOS NUESTRO DATAFRAME CON LOS DATOS GEOGRÁFICOS
   df=pd.merge(df,geograficosMUN,on="cveunica",how="right")
   print(df)
   #LLENAMOS LOS NA DE LAS FILAS DE MORBILIDAD SI NO CON 0
   df.iloc[ : , 4]= df.iloc[ : , 4].fillna(0)
   df.iloc[ : , 5]= df.iloc[ : , 5].fillna(0)
   #df[1] = df[1].fillna(0)
   #df[2] = df[2].fillna(0)
   #RENOMBRAMOS COLUMNAS DE LOS CASOS SI/NO
   df =df.rename(columns={1:str(i),2:"no_"+str(i),97:"noaplica_"+str(i),98:"seignora_"+str(i),99:"noespecificado"})
   #INICIAMOS LOOP PARA CADA COLUMNA DE LOS CASOS DE MORBILIDAD SI, NO,
   for j in df.columns[4:6]:
       #df[j] = df[j].fillna(0)
       labels=["1","2","3","4","5","6","7","8","9","10"]
       df["ordenvalido"]="Confirmados"
       df["clasevalida"]="Datos_abiertos"
       df["familiavalida"]="Incidencias"
       df["diacolecta"]="1"
       df["mescolecta"]="1"
       df["aniocolecta"]="2010"
       df["idejemplar"]="COVID-19"
       df["generovalido"]="confirmado_"+(j)
       df["paismapa"]="México"
       df["altitudmapa"]="9999"
       df["reinovalido"]="Epidemiologicos"
       df["phylumdivisionvalido"]="Dirección General de Epidemiología"
       df["categoriainfraespecievalida"]="total__municipio_confirmados_"+(j)
       df["proyecto"]="EPI-SPECIES"
       df["valores"]=df[j]
       df["calculoespecievalida"]=pd.cut(df["valores"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
       df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
       df["especievalida"] = df["especievalida"].str.replace(',',':')
       df["especievalida"] = df["especievalida"].str.replace(': ',':')
       df["especievalida"] = df["especievalida"].str.replace('(','')
       df["especievalida"] = df["especievalida"].str.replace(']','')
       df["especievalida"] = df["especievalida"].str.replace(')','')
       df["especievalida"] = df["especievalida"].str.replace('[','')
       #GUARDAMOS NUESTRO DATAFRAME SELECCIONANDO LAS VARIABLES DE INTERÉS
       dfmunicipios=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
       #NOMBRAMOS CADA DATAFRAME
       dfmunicipios.to_csv("datosMPO" +"_confirmados_"+ str(j) + ".csv",encoding="UTF-8",index=False)
       #DESCARGAMOS CADA DATAFRAME
       files.download("datosMPO" +"_confirmados_"+ str(j) + ".csv")
       df["generovalido"]="confirmado_"+(j)+"_por100,000hab"
       df["valores"]=df[j]/df["Población"]*100000
       df["calculoespecievalida"]=pd.cut(df["valores"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
       df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
       df["especievalida"] = df["especievalida"].str.replace(',',':')
       df["especievalida"] = df["especievalida"].str.replace(': ',':')
       df["especievalida"] = df["especievalida"].str.replace('(','')
       df["especievalida"] = df["especievalida"].str.replace(']','')
       df["especievalida"] = df["especievalida"].str.replace(')','')
       df["especievalida"] = df["especievalida"].str.replace('[','')
       dfmunckhab=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
       dfmunckhab.to_csv("datosMPO" +"_confirmados_"+ str(j) +"p100kh"+".csv",encoding="UTF-8",index=False)
       #DESCARGAMOS CADA DATAFRAME
       files.download("datosMPO" +"_confirmados_"+ str(j) +"p100kh"+".csv")


decesos
     entidad_res municipio_res  ... cve_municipio COVID      municipio COVID
0             01           001  ...                   1       Aguascalientes
1             01           002  ...                   2             Asientos
2             01           005  ...                   5          Jesús María
3             01           006  ...                   6  Pabellón de Arteaga
4             01           007  ...                   7      Rincón de Romos
...          ...           ...  ...                 ...                  ...
2451         NaN           NaN  ...                  10             Calakmul
2452         NaN           NaN  ...                   9            Escárcega
2453         NaN           NaN  ...                   8               Tenabo
2454         NaN           NaN  ...                   6            Hopelchén
2455         NaN           NaN  ...                   4            Champotón

[2456 rows x 16 columns]


In [26]:
#LOOP ESPECIES  COMORBILIDAD  ESTADOS TOTALES COMENTADO PARA TODAS LAS VARIABLES MORBILIDAD CONFIRMADOS COVID
for i in datosfederalCOVID19.columns[20:35]:
   print (i)
   #HACEMOS LA AGRUPACION POR ENTIDAD,RESULTADO Y CONTAMOS POR LA COLUMNA DE MORBILIDAD
   df=datosfederalCOVID19.groupby(["entidad_res","resultado"])[(i)].value_counts().unstack().fillna(0)
   #CONVERTIMOS A UN DATAFRAME LOS RESULTADOS DEL GROUPBY
   df.reset_index(inplace=True)
   #AQUI SELECCIONAMOS LOS CONFIRMADOS O LOS QUE ESTÁN EN ESPERA DE ACUERDO AL RESULTADO DE LA PRUEBA
   df=df.loc[df.loc[:,"resultado"] == 1]
   #JUNTAMOS NUESTRO DATAFRAME CON LOS DATOS GEOGRÁFICOS
   df=pd.merge(df,geograficosEDO,on="entidad_res",how="right")
   print(df)
   #LLENAMOS LOS NA DE LAS FILAS DE MORBILIDAD SI NO CON 0
  # df[1] = df[1].fillna(0)
  # df[2] = df[2].fillna(0)
   df.iloc[ : , 2]= df.iloc[ : , 2].fillna(0)
   df.iloc[ : , 3]= df.iloc[ : , 3].fillna(0)
   #RENOMBRAMOS COLUMNAS DE LOS CASOS SI/NO
   df =df.rename(columns={1:str(i),2:"no_"+str(i),97:"noaplica_"+str(i),98:"seignora_"+str(i),99:"noespecificado"})
   #INICIAMOS LOOP PARA CADA COLUMNA DE LOS CASOS DE MORBILIDAD SI, NO,
   for j in df.columns[2:4]:
       labels=["1","2","3","4","5","6","7","8","9","10"]
       df["ordenvalido"]="Confirmados"
       df["clasevalida"]="Datos_abiertos"
       df["familiavalida"]="Incidencias"
       df["diacolecta"]="1"
       df["mescolecta"]="1"
       df["aniocolecta"]="2010"
       df["idejemplar"]="COVID-19"
       df["generovalido"]="confirmado_"+(j)+"_por100,000hab"
       df["paismapa"]="México"
       df["altitudmapa"]="9999"
       df["reinovalido"]="Epidemiologicos"
       df["phylumdivisionvalido"]="Dirección General de Epidemiología"
       df["categoriainfraespecievalida"]="total__estado_confirmados_"+(j)
       df["proyecto"]="EPI-SPECIES"
       df["valores"]=df[j]
       df["calculoespecievalida"]=pd.cut(df["valores"],8)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
       df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
       df["especievalida"] = df["especievalida"].str.replace(',',':')
       df["especievalida"] = df["especievalida"].str.replace(': ',':')
       df["especievalida"] = df["especievalida"].str.replace(': ',':')
       df["especievalida"] = df["especievalida"].str.replace('(','')
       df["especievalida"] = df["especievalida"].str.replace(']','')
       df["especievalida"] = df["especievalida"].str.replace(')','')
       df["especievalida"] = df["especievalida"].str.replace('[','')
       #GUARDAMOS NUESTRO DATAFRAME SELECCIONANDO LAS VARIABLES DE INTERÉS
       df2=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
       #NOMBRAMOS CADA DATAFRAME
       df2.to_csv("datosEDO" +"_confirmados_"+ str(j)+".csv",encoding="UTF-8",index=False)
       #DESCARGAMOS CADA DATAFRAME
       files.download("datosEDO" +"_confirmados_"+ str(j)+".csv")
       df["generovalido"]="confirmado_"+(j)+"_por100,000hab"
       df["valores"]=df[j]/df["Población"]*100000
       df["calculoespecievalida"]=pd.cut(df["valores"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES     
       df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
       df["especievalida"] = df["especievalida"].str.replace(',',':')
       df["especievalida"] = df["especievalida"].str.replace(': ',':')
       df["especievalida"] = df["especievalida"].str.replace('(','')
       df["especievalida"] = df["especievalida"].str.replace(']','')
       df["especievalida"] = df["especievalida"].str.replace(')','')
       df["especievalida"] = df["especievalida"].str.replace('[','')
       dfedoc100khab=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
       dfedoc100khab.to_csv("datosEDO" +"_confirmados_"+ str(j) +"p100kh"+".csv",encoding="UTF-8",index=False)
       #DESCARGAMOS CADA DATAFRAME
       files.download("datosEDO" +"_confirmados_"+ str(j) +"p100kh"+".csv")


intubado
   entidad_res  resultado      1  ...    latitud    longitud  Población
0           01          1    1.0  ...  21.811436 -102.295803    1184996
1           02          1   33.0  ...  30.036510 -114.957214    3155070
2           03          1    1.0  ...  25.491348 -111.820079     637026
3           04          1    0.0  ...  18.402829  -89.661574     822441
4           05          1    4.0  ...  27.122395 -101.245782    3406465
5           06          1    0.0  ...  19.009246 -103.998420    8851080
6           07          1    2.0  ...  15.414140  -92.681685    2748391
7           08          1   10.0  ...  30.449237 -106.380692     650555
8           09          1  128.0  ...  19.426340  -99.126612    4796580
9           10          1    1.0  ...  24.511116 -104.935886    1632934
10          11          1    5.0  ...  20.533231 -101.536012    5486372
11          12          1   10.0  ...  16.957200  -99.714309    3388768
12          13          1    9.0  ...  20.206424  -98.4

In [0]:
#ESPECIE TOTAL PRUEBAS POSITIVO COVID MUNICIPIO
df=datosfederalCOVID19.groupby(["entidad_res","municipio_res","cveunica"])[("resultado")].value_counts().unstack().fillna(0)
df[1]= df[1].fillna(0)
df[2]= df[2].fillna(0)
df[3]= df[3].fillna(0)
df["PruebasPOSITIVO"]=df[1]
df.reset_index(inplace=True)
df=pd.merge(df,geograficosMUN,on="cveunica",how="right")
df[1]= df[1].fillna(0)
df[2]= df[2].fillna(0)
df[3]= df[3].fillna(0)
df["PruebasPOSITIVO"]=df["PruebasPOSITIVO"].fillna(0)
labels=["1","2","3","4","5","6","7","8","9","10"]
df["ordenvalido"]="Totales"
df["clasevalida"]="Datos_abiertos"
df["familiavalida"]="Incidencias"
df["diacolecta"]="1"
df["mescolecta"]="1"
df["aniocolecta"]="2010"
df["idejemplar"]="COVID-19"
df["generovalido"]="PruebasPOSITIVOpmunicipio"
df["paismapa"]="México"
df["altitudmapa"]="9999"
df["reinovalido"]="Epidemiologicos"
df["phylumdivisionvalido"]="Dirección General de Epidemiología"
df["categoriainfraespecievalida"]="total_municipio_pruebaspositivo"
df["proyecto"]="EPI-SPECIES"
df["calculoespecievalida"]=pd.cut(df["PruebasPOSITIVO"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
       #GUARDAMOS NUESTRO DATAFRAME SELECCIONANDO LAS VARIABLES DE INTERÉS
dfmunicipios=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
      #NOMBRAMOS CADA DATAFRAME
dfmunicipios.to_csv("datosMPOpruebaspositivo.csv",encoding="UTF-8",index=False)
       #DESCARGAMOS CADA DATAFRAME
files.download("datosMPOpruebaspositivo.csv")
df["generovalido"]="pruebaspositivopmunicipiop100,000hab"
df["valores"]=df["PruebasPOSITIVO"]/df["Población"]*100000
df["calculoespecievalida"]=pd.cut(df["valores"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
dfmunckhab=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
dfmunckhab.to_csv("datosMPOpruebaspositivop100kh.csv",encoding="UTF-8",index=False)
#DESCARGAMOS CADA DATAFRAME
files.download("datosMPOpruebaspositivop100kh.csv")

In [0]:
#ESPECIE TOTAL PRUEBAS POSITIVO COVID ESTADO
df=datosfederalCOVID19.groupby(["entidad_res"])[("resultado")].value_counts().unstack().fillna(0)
df[1]= df[1].fillna(0)
df[2]= df[2].fillna(0)
df[3]= df[3].fillna(0)
df["PruebasPOSITIVO"]=df[1]
df.reset_index(inplace=True)
df=pd.merge(df,geograficosEDO,on="entidad_res",how="right")
df[1]= df[1].fillna(0)
df[2]= df[2].fillna(0)
df[3]= df[3].fillna(0)
df["PruebasPOSITIVO"]=df["PruebasPOSITIVO"].fillna(0)
labels=["1","2","3","4","5","6","7","8","9","10"]
df["ordenvalido"]="Totales"
df["clasevalida"]="Datos_abiertos"
df["familiavalida"]="Incidencias"
df["diacolecta"]="1"
df["mescolecta"]="1"
df["aniocolecta"]="2010"
df["idejemplar"]="COVID-19"
df["generovalido"]="PruebasPOSITIVOestado"
df["paismapa"]="México"
df["altitudmapa"]="9999"
df["reinovalido"]="Epidemiologicos"
df["phylumdivisionvalido"]="Dirección General de Epidemiología"
df["categoriainfraespecievalida"]="total_estado_pruebaspositivo"
df["proyecto"]="EPI-SPECIES"
       #df["valores"]=df[j]
df["calculoespecievalida"]=pd.cut(df["PruebasPOSITIVO"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
#print(df["Pruebastotales"].hist()
       #GUARDAMOS NUESTRO DATAFRAME SELECCIONANDO LAS VARIABLES DE INTERÉS
dfmunicipios=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
      #NOMBRAMOS CADA DATAFRAME
dfmunicipios.to_csv("datosEDOpruebaspositivoconfirmados.csv",encoding="UTF-8",index=False)
       #DESCARGAMOS CADA DATAFRAME
files.download("datosEDOpruebaspositivoconfirmados.csv")
df["generovalido"]="pruebaspositivoedop100,000hab"
df["valores"]=df["PruebasPOSITIVO"]/df["Población"]*100000
df["calculoespecievalida"]=pd.cut(df["valores"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
dfmunckhab=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
dfmunckhab.to_csv("datosEDOpruebaspositivop100kh.csv",encoding="UTF-8",index=False)
#DESCARGAMOS CADA DATAFRAME
files.download("datosEDOpruebaspositivop100kh.csv")



In [0]:
#ESPECIE TOTAL HOSPITALIZADOS POSITIVO COVID MUNICIPIO
#CALCULO VFhospitalizados municipio= resultado1,y tipo paciente 2
df=datosfederalCOVID19.groupby(["entidad_res","municipio_res","cveunica","resultado"])[("tipo_paciente")].value_counts().unstack().fillna(0)
df.reset_index(inplace=True)
df=df.loc[df.loc[:,"resultado"] == 1]
df[2]= df[2].fillna(0)
df=pd.merge(df,geograficosMUN,on="cveunica",how="right")
df["resultado"]= df["resultado"].fillna(0)
df[2]= df[2].fillna(0)
labels=["1","2","3","4","5","6","7","8","9","10"]
df["hospitalizados"]= df[2]
df["ordenvalido"]="Totales"
df["clasevalida"]="Datos_abiertos"
df["familiavalida"]="Incidencias"
df["diacolecta"]="1"
df["mescolecta"]="1"
df["aniocolecta"]="2010"
df["idejemplar"]="COVID-19"
df["generovalido"]="HospitalizadosCOVIDpmunicipio"
df["paismapa"]="México"
df["altitudmapa"]="9999"
df["reinovalido"]="Epidemiologicos"
df["phylumdivisionvalido"]="Dirección General de Epidemiología"
df["categoriainfraespecievalida"]="total_municipio_COVID_hospitalizados"
df["proyecto"]="EPI-SPECIES"
df["calculoespecievalida"]=pd.cut(df["hospitalizados"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
       #GUARDAMOS NUESTRO DATAFRAME SELECCIONANDO LAS VARIABLES DE INTERÉS
dfmunicipios=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
      #NOMBRAMOS CADA DATAFRAME
dfmunicipios.to_csv("datosMPOhospitalizadosCOVID.csv",encoding="UTF-8",index=False)
       #DESCARGAMOS CADA DATAFRAME
files.download("datosMPOhospitalizadosCOVID.csv")
df["generovalido"]="datosMPOhospitalizadosCOVIDpor100,000hab"
df["valores"]=df["hospitalizados"]/df["Población"]*100000
df["calculoespecievalida"]=pd.cut(df["valores"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
dfmunckhab=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
dfmunckhab.to_csv("datosMPOhospitalizadosCOVIDpor100,000hab.csv",encoding="UTF-8",index=False)
#DESCARGAMOS CADA DATAFRAME
files.download("datosMPOhospitalizadosCOVIDpor100,000hab.csv")

In [0]:
##ESPECIE TOTAL HOSPITALIZADOS POSITIVO COVID ESTADO
  #CALCULO VFhospitalizadosEstado=  resultado1 y tipo paciente 2
df=datosfederalCOVID19.groupby(["entidad_res","resultado"])[("tipo_paciente")].value_counts().unstack().fillna(0)
df.reset_index(inplace=True)
df=df.loc[df.loc[:,"resultado"] == 1]
df[2]= df[2].fillna(0)
df=pd.merge(df,geograficosEDO,on="entidad_res",how="right")
df["resultado"]= df["resultado"].fillna(0)
df[2]= df[2].fillna(0)  
labels=["1","2","3","4","5","6","7","8","9","10"]
df[2]= df[2].fillna(0)
df["hospitalizados"]=df[2]
df["ordenvalido"]="Totales"
df["clasevalida"]="Datos_abiertos"
df["familiavalida"]="Incidencias"
df["diacolecta"]="1"
df["mescolecta"]="1"
df["aniocolecta"]="2010"
df["idejemplar"]="COVID-19"
df["generovalido"]="HospitalizadosCOVIDtotalespestado"
df["paismapa"]="México"
df["altitudmapa"]="9999"
df["reinovalido"]="Epidemiologicos"
df["phylumdivisionvalido"]="Dirección General de Epidemiología"
df["categoriainfraespecievalida"]="total_estado_COVID_hospitalizados"
df["proyecto"]="EPI-SPECIES"
       #df["valores"]=df[j]
df["calculoespecievalida"]=pd.cut(df["hospitalizados"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
#print(df["Pruebastotales"].hist()
       #GUARDAMOS NUESTRO DATAFRAME SELECCIONANDO LAS VARIABLES DE INTERÉS
dfEDO=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
      #NOMBRAMOS CADA DATAFRAME
dfEDO.to_csv("datosEDOhospitalizadosCOVID.csv",encoding="UTF-8",index=False)
       #DESCARGAMOS CADA DATAFRAME
files.download("datosEDOhospitalizadosCOVID.csv")
df["generovalido"]="HospitalizadosCOVIDpestado100,000hab"
df["valores"]=df["hospitalizados"]/df["Población"]*100000
df["calculoespecievalida"]=pd.cut(df["valores"],10)
       #CONSTRUCCION DEL CAMPO ESPECIE VALIDA CON DOS CAMPOS Y QUITANDO CARACTERES
df["especievalida"]=df['generovalido'].str.cat(df['calculoespecievalida'].values.astype(str),sep=" ")
df["especievalida"] = df["especievalida"].str.replace(',',':')
df["especievalida"] = df["especievalida"].str.replace(': ',':')
df["especievalida"] = df["especievalida"].str.replace('(','')
df["especievalida"] = df["especievalida"].str.replace(']','')
df["especievalida"] = df["especievalida"].str.replace(')','')
df["especievalida"] = df["especievalida"].str.replace('[','')
dfEDOkhab=df.loc[:, ['estadomapa','ordenvalido','clasevalida','familiavalida','diacolecta','mescolecta','aniocolecta','idejemplar','generovalido','paismapa','altitudmapa','reinovalido','phylumdivisionvalido','categoriainfraespecievalida','proyecto','especievalida','latitud','longitud']]
dfEDOkhab.to_csv("datosEDOHospitalizadosCOVIDp100kh.csv",encoding="UTF-8",index=False)
#DESCARGAMOS CADA DATAFRAME
files.download("datosEDOHospitalizadosCOVIDp100kh.csv")